<a href="https://colab.research.google.com/github/JoseFPortoles/Notes-on-Constraints-Algorithms/blob/master/Notes_on_Search_Algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Search tree algorithms

## Branch and bound, Extension Filtering, Admissible Heuristic, $A^*$ and Consistency
The problem is to find the best path (lower accumulated traversing cost) in a graph from a starting node to a goal node.

In branch and bound we keep a queue of paths that is initialised by a single path containing the root node. Keep extending the paths, removing paths with loops and sorting the queue according to accumulated cost of the path in such a way that the "shortest" path gets extended on the next iteration.

**Extended node filtering:**
A more efficient way of searching for the best path is adding an extended node list to the branch and bound algorithm. If a path in the queue terminates at a node that has already been extended before (meaning that it has been reached by a better path before given the B&B queue dynamic), that path is not extended, just removed from the queue.

**Admissible heuristic:**
Another possible improvement respect to B&B is the use of a heuristic measure h that gives a lower bound of the remaining distance (cost) to the goal. The queue then is sorted on each iteration according to the sum of accumulated cost and estimated remaining cost (cost(Node) + h(Node)). Once the goal node is found (path B), any other path whose cost+h is higher than cost(B) is known to be worse than B (given that h is a lower bound to remaining cost, so the actual cost of the path to goal is going to be even higher) and therefore not worth extending.

**${A^*}$ algorithm:**
Since the two modifications explained above seem to have advantages, there is the possibility to combine them and see if it yields further improvements. $A^*$ is then a Branch and Bound algorithm with extended node filtering and admissible heuristic of the remaining cost to goal.

**Consistency:**
Branch and bound and extended node filtering provide optimal search improvements over blind search counterparts such as Depth First Search (DFS) and Breadth First Search (BFS). However, the use of an admissible heuristic and the $A^*$ algorithm, while advantageous in maps, where the positions of the graph nodes reflect euclidian distances, can end up in missing the best path in graphs that are not maps. In these cases it still is possible to use a heuristic that is consistent rather than simply admissible. Consistency is:

\begin{equation}
\left|H(x,G)-H(y,G)\right| \leq D(x,y)
\end{equation}

This is, for any pair of nodes x,y, the absolute value of the difference between estimated distances H to goal G from node x and from node y is less or equal to the actual distance between x and y.







A*

* Start with only the root node in the queue
* While:
  * Queue is not empty and
  * Path to goal node not in the queue
    * Remove the first path in the queue
    * Expand it to all its children
    * Discard all the extended paths with loops
    * Insert the remaining expanded paths in the queue
    * Sort the queue according to f = cost + h where:
      * **cost** : is the accumulated cost of the path
      * **h**: is an estimation of remaining distance to the goal from the end of the path
    * If in the queue there is a path P ending in node I and another path Q containing node I and $cost\left(P\right)\geq cost\left(Q\right)$ then remove P from the queue  
* If the queue is empty, stop and declare failure
* If a path to the goal is in the queue stop and declare success



### Some properties of $A^*$
If there is one or more paths to the goal, one of them will be the best path B. Necessarily the initial path in the queue, the one formed by the root node alone, will be a subpath of B. If we expand that initial path to the children nodes, as in the first iteration of the WHILE loop, then at least one of the expanded paths will be a subpath of B. Whenever there is a subpath of B in the queue, its expansion will result in at least one new subpath of B being added to the queue. That means that there is always a subpath of B in the queue.



# Constraints

## Numerical constraint networks
Any system of equations define a numerical constraint net with variables boxes and operator boxes with links between both. 

In a numerical constraint net, values assigned to the variables can flow in different directions. The flow of data in such a net can proceed in different directions, in the same ways as mathematical equations which don't impose a direction in how the equation is used procedurally.

### Spreadsheets
Spreadsheets are an example of application of numerical constraint networks. Typically the same spreadsheet can be used in two modes:
* **1st Mode**: Define constants, variables and equations linking them
* **2nd Mode** (What if mode): Available values for some variables propagates through the net in order to find the values of the dependent variables

This is similar but weaker than constraint propagation, given that values cannot flow in different directions in a spreadsheet. A cell that is already occupied by an expression cannot be assigned a value without removing the expression.

### More advanced numerical propagation
If constraints are defined on the possible values of some or all of the variables in a numerical constraint net, some of the possible values in a variable could be inconsistent with some of the possible values that other variables in the net can take. Checking for consistency in this context consist in assume one of the possible values $x$ for the variable $V$ and propagate it through the net, checking if the values propagated to each of the other variables is compatible with the corresponding sets of possible values. If inconsistent then $x$ is discarded as a possible value for $V$. When a set of consistent values is found, that is a solution.

### Initial conclusions
The most important element in the solution of a problem in software or knowledge engineering is the recognition, representation, exploitation and propagation of the natural constraints of the problem.

In the rest of the notebook, after introducing a fairly generic class of constraint problems and a set of methods for propagation in the context of such problems:
* Variants of standard backtracking
* Relaxation (arc-consistency) techniques: These are not complete, as are not always able to produce a solution but still useful by helping reduce the number of possibilities
* Backtracking + Relaxation techniques: mostly in scheduling, rostering and planning problems

Then some applications will be discussed like the problem of producing a 3D interpretation of a 2D drawing, in particular Waltz algorithm. Also the problem of understanding the meaning of natural language processing 

As a final conclusion, point out that Artificial Neural Networks can be considered a form of numerical constraint propagation network. Here, numerical values at the input are propagated forward through a set of connected multiply and adder operator boxes.

## Constraint problem solving: Definitions
### **Constraint problem (consistent labelling problem)** consists of:
* A finite set of variables $\{z_1, z_2 \dots z_n\}$
* A domain $d_i$ (set of possible values) for each variable $z_i$
* For each two variables a constraint (a relation $c\left(z_i,z_j\right)$ )

A solution for the problem is the selection of a single value of each domain such that all the constraints are met.

Depending on the problem we might be happy obtaining a single solution, or we might want to obtain all solutions to the problem. In other cases we might want to identify an optimal solution, understood as a solution that maximises a certain function defined on the variables.

The above definitions contain some restrictions that could be relaxed for certain types of problems. For instance constraints could be defined among three or more variables while the above definition focuses on ***binary  constraints***. This is done only for the sake of notational simplicity. The techniques for dealing with binary constraint problems can, with more or less difficulty, generalised to constraints over 3 or more variables.

Another restriction is the assumption of finite domains. This reduces the scope of the techniques considered to finite domain constraint problems, which make those techniques not easily generalisable to infinite discrete or continuous domains. In particular the backtracking algorithms have no counterparts in infinite domains.




















## Examples
Now let's have a look at some examples of constraint problems with some "naive" solutions in python

### Example q-queens
Given an integer number q, how to place q queens in a $q\times{q}$ chess board in such a way that don't attack each other.

#### $1^{st}$ approach
* One variable per each queen
* Same domain for all variables $\{\left(n,m\right) \text{for n,m in } \{1,\ldots q\}\}$ (n rows and m columns of the board)
* Constraints: $C\left(z_i,z_j\right)\,=\,\,row\left(z_i\right)\ne row\left(z_j\right)\land col\left(z_i\right)\ne col\left(z_j\right) \land \|row\left(z_i\right)-row\left(z_j\right)\| \ne \|col\left(z_i\right)-col\left(z_j\right)\|$


In [ ]:
# A solution for the q-queens for q = 4  
def board(q):
  '''Takes a solution and display the board as 1s (queens) 
  and 0s (empty squares)'''
  board = ""
  for i in range(1,5):
    for j in range(1,5):
      if (i,j) in q:
        board = board + "1 "
      else:
        board = board + "0 "
    board = board + "\n"
  return board

# Domain
D = [(n,m) for n in range(1,5) for m in range(1,5)]

# Solution list:
# Start by assigning entire domain to each queen variable
S = [(q1,q2,q3,q4) for q1 in D for q2 in D for q3 in D for q4 in D]

# Constraints 1 and 2: No two queens with same rows or columns 
S = [(q1,q2,q3,q4) for (q1,q2,q3,q4) in S 
     if len(set([q1[0],q2[0],q3[0],q4[0]])) == len([q1[0],q2[0],q3[0],q4[0]]) 
     and len(set([q1[1],q2[1],q3[1],q4[1]])) == len([q1[1],q2[1],q3[1],q4[1]])
     ]

# Constraint 3: No two queens on the same diagonal
S = [(q1,q2,q3,q4) for (q1,q2,q3,q4) in S if abs(q1[0]-q2[0])!=abs(q1[1]-q2[1])]
S = [(q1,q2,q3,q4) for (q1,q2,q3,q4) in S if abs(q1[0]-q3[0])!=abs(q1[1]-q3[1])]
S = [(q1,q2,q3,q4) for (q1,q2,q3,q4) in S if abs(q1[0]-q3[0])!=abs(q1[1]-q3[1])]
S = [(q1,q2,q3,q4) for (q1,q2,q3,q4) in S if abs(q1[0]-q4[0])!=abs(q1[1]-q4[1])]
S = [(q1,q2,q3,q4) for (q1,q2,q3,q4) in S if abs(q2[0]-q3[0])!=abs(q2[1]-q3[1])]
S = [(q1,q2,q3,q4) for (q1,q2,q3,q4) in S if abs(q2[0]-q4[0])!=abs(q2[1]-q4[1])]
S = [(q1,q2,q3,q4) for (q1,q2,q3,q4) in S if abs(q3[0]-q4[0])!=abs(q3[1]-q4[1])]

# Sort each solution to identify repeats
S = [tuple(sorted(q)) for q in S]

# Remove repeats
S = set(S)

# Display all solutions 
for s in S:
  print(board(s))

0 1 0 0 
0 0 0 1 
1 0 0 0 
0 0 1 0 

0 0 1 0 
1 0 0 0 
0 0 0 1 
0 1 0 0 



#### 2nd Approach
Each queen in a valid solution is going to be on a separate row. So one queen $q_1$ for row 1, another $q_2$ for row 2 and so on.

We can then assign a variable to each row that records the position of the corresponding queen inside the row. So we have variables $z_1$ to $z_q$.

* For queen $q_i$ on row $i$: Variable $z_i$ 
* For each $z_i$ domain $\{1,\ldots ,q\}$
* Constraints: $c\left(z_i,z_j\right)\,=\,z_i\neq z_j \land \|z_i-z_j\|\neq\|i-j\|$


In [ ]:
def board(q):
  '''Takes a solution and display the board as 1s (queens) 
  and 0s (empty squares)'''
  board = ""
  for i in range(len(q)):
    for j in range(len(q)):
      if j == q[i]-1:
        board = board + "1 "
      else:
        board = board + "0 "
    board = board + "\n"
  return board

D = [x for x in range(1,5)]

S = [(z1,z2,z3,z4) for z1 in range(1,5) for z2 in range(1,5) 
    for z3 in range(1,5) for z4 in range(1,5)
    ]

# Constraint 1: zi!=zj for all i,j
S = [z for z in S if len(z)==len(set(z))]
# Constraint 2: |zi-zj| != |i-j|
def constraint2(z):
  for i in range(len(z)-1):
    for j in range(i+1,len(z)):
      if abs(z[i]-z[j])==abs(i-j):
        return False 
  return True
S = filter(constraint2, S)
S = set(S)
for s in S:
  print(board(s))

0 0 1 0 
1 0 0 0 
0 0 0 1 
0 1 0 0 

0 1 0 0 
0 0 0 1 
1 0 0 0 
0 0 1 0 



### Confused q-queens

This is exactly the same problems except that in a solution the queens need to be in positions were each two queens attack each other. For q = 4 we copy the first code and modify it accordingly. 

In [ ]:
# A solution for the q-queens for q = 4  
def board(q):
  '''Takes a solution and display the board as 1s (queens) 
  and 0s (empty squares)'''
  board = ""
  for i in range(1,5):
    for j in range(1,5):
      if (i,j) in q:
        board = board + "1 "
      else:
        board = board + "0 "
    board = board + "\n"
  return board

# Domain
D = [(n,m) for n in range(1,5) for m in range(1,5)]

# Solution list:
# Start by assigning entire domain to each queen variable
S = [(q1,q2,q3,q4) for q1 in D for q2 in D for q3 in D for q4 in D]

# Constraints: Queens need to be either in the same row or the same column or the same diagonal

def cqqueensolution(s):
  for i in range(len(s)-1):
    row_si = s[i][0]
    col_si = s[i][1]
    for j in range(i+1,len(s)):
      row_sj = s[j][0]
      col_sj = s[j][1]
      if len(s) != len(set(s)):
        return False
      if abs(row_si-row_sj) != abs(col_si-col_sj) and row_si != row_sj and col_si != col_sj:
        return False
  return True


# Sort each solution to identify repeats
S = filter(cqqueensolution,S)
S = [tuple(sorted(s)) for s in S]
S = set(S)
print("Number of solutions: "+str(len(S)))

# Display all solutions 
for s in S:
  print(board(s))

Number of solutions: 68
0 0 0 0 
1 0 0 0 
1 1 0 0 
1 0 0 0 

1 0 1 0 
0 1 0 0 
0 0 1 0 
0 0 0 0 

0 0 1 0 
0 1 0 1 
0 0 1 0 
0 0 0 0 

0 1 1 0 
0 1 1 0 
0 0 0 0 
0 0 0 0 

0 0 0 0 
0 0 0 0 
1 1 1 0 
0 1 0 0 

0 0 0 0 
0 1 0 0 
1 1 0 0 
0 1 0 0 

0 0 0 0 
1 0 1 0 
0 0 0 0 
1 0 1 0 

1 0 0 0 
0 1 0 0 
1 0 1 0 
0 0 0 0 

0 0 1 1 
0 0 1 1 
0 0 0 0 
0 0 0 0 

0 0 0 0 
0 0 0 0 
0 0 1 1 
0 0 1 1 

0 0 0 0 
0 0 0 1 
0 0 1 0 
0 1 0 1 

0 0 0 0 
0 0 0 0 
0 0 1 0 
0 1 1 1 

0 1 0 0 
0 0 1 0 
0 1 0 1 
0 0 0 0 

0 0 0 0 
1 0 0 0 
0 1 0 0 
1 0 1 0 

0 1 0 1 
0 0 1 0 
0 0 0 1 
0 0 0 0 

0 1 0 0 
0 1 1 0 
0 1 0 0 
0 0 0 0 

1 1 1 1 
0 0 0 0 
0 0 0 0 
0 0 0 0 

0 1 0 0 
1 1 1 0 
0 0 0 0 
0 0 0 0 

0 0 0 0 
1 1 1 1 
0 0 0 0 
0 0 0 0 

0 0 0 0 
1 0 1 0 
0 1 0 0 
0 0 1 0 

0 0 0 1 
0 0 1 1 
0 0 0 1 
0 0 0 0 

1 1 0 0 
1 1 0 0 
0 0 0 0 
0 0 0 0 

0 0 0 0 
0 1 1 0 
0 1 1 0 
0 0 0 0 

0 0 0 1 
0 0 1 0 
0 1 0 0 
1 0 0 0 

0 0 0 0 
0 1 0 0 
1 0 1 0 
0 1 0 0 

0 0 0 0 
0 0 0 0 
0 1 1 1 
0 0 1 0 

0 0 0 0 
0 1 0

Notice that the above code follows the first representation, namely one variable per queen, each variable containing a tuple (n,m) with the position of the queen. The 2$^{nd}$ representation was introduced because, for the q-queens problem it didn't make sense to consider solutions in which two queens shared the same row. In this way this constraint was buil into the representation reducing the amount of computation.

We can adopt the 2$^{nd}$ representation in the solution of the confused q-queens problem, although in doing so we will be formulating a different problem, as many of the solutions found by the previous code will no longer be solutions of the problem. This version of the confused q-queens problem has always $q+2$ solutions, corresponding to all the queens aligned in the same column or along one of the two diagonals of the board. 

In [ ]:
def board(q):
  '''Takes a solution and display the board as 1s (queens) 
  and 0s (empty squares)'''
  board = ""
  for i in range(len(q)):
    for j in range(len(q)):
      if j == q[i]-1:
        board = board + "1 "
      else:
        board = board + "0 "
    board = board + "\n"
  return board

D = [x for x in range(1,5)]

S = [(z1,z2,z3,z4) for z1 in range(1,5) for z2 in range(1,5) 
    for z3 in range(1,5) for z4 in range(1,5)
    ]

# Constraint: two queens are either in the same column or in the same diagonal 
def constraint2(z):
  for i in range(len(z)-1):
    for j in range(i+1,len(z)):
      if abs(z[i]-z[j])!=abs(i-j) and z[i]!=z[j]:
        return False 
  return True

S = filter(constraint2, S)

S = set(S)

for s in S:
  print(board(s))

1 0 0 0 
0 1 0 0 
0 0 1 0 
0 0 0 1 

0 1 0 0 
0 1 0 0 
0 1 0 0 
0 1 0 0 

1 0 0 0 
1 0 0 0 
1 0 0 0 
1 0 0 0 

0 0 0 1 
0 0 0 1 
0 0 0 1 
0 0 0 1 

0 0 0 1 
0 0 1 0 
0 1 0 0 
1 0 0 0 

0 0 1 0 
0 0 1 0 
0 0 1 0 
0 0 1 0 



## Search representations

### OR-TREE

Here the search space is represented by a tree. 
 

1.   From the root node branches to all the values of the domain for $z_1$.
2.   Each of these initial $z_1$ branches are then extended to all the values in the domain of $z_2$.
3. On each of these branches we verify the constraint $C\left(z_1,z_2\right)$. The verification results in either True (V) or False (X).
4. Only the True (V) branches meet the constraint $C\left(z_1,z_2\right)$, therefore, only those nodes are further extended to all the values in the domain of $z_3$.
5. In this new level of the tree both the constraints $C\left(z_1,z_3\right)$ and $C\left(z_2,z_3\right)$ are verified.
6. Only the leave nodes that verify as V for both constraints are further extended.
7. The procedure continues until variable $z_n$ including constraint tests $C\left(z_1,z_n\right)$ to $C\left(z_{n-1},z_n\right)$
8. Those final leaf nodes for which all the constraint tests are passed correspond to the different solutions to the problem. By combining the values of the variables $z_i$ along the paths to each of these final leaf nodes the solutions get represented. 

   

![OR-Tree](https://raw.githubusercontent.com/JoseFPortoles/Notes-on-Constraints-Algorithms/master/grafos/ortree.png)


In the figure above an example is shown of an OR-tree for a problem with three variables $z_1$ to $z_3$ and binary constraints between them.

It has to be pointed out that the OR-tree is simply a representation of the search space, not a search algorithm. Different search algorithms can be implemented on top of it (breadth-first, depth-first, random-search, etc...).

### The Network Representation

This is an alternative representation of the search space as a non-directed graph with the variables and their domains at the nodes and the constraints represented by edges.

This representation is meant to be used in conjunction with 'relaxation' or 'arc consistency' methods.

![Network Representation Graph](https://raw.githubusercontent.com/JoseFPortoles/Notes-on-Constraints-Algorithms/master/grafos/netrep.png)

Relaxation proceeds by:
* Selecting a variable $z_i$ and a particualr value $a_{ij}$ inside its domain.
* Select any constraint involving $z_i$ (any arc reaching $z_i$ in the above representation), for instance $C\left(z_i,z_k\right)$.
* Check wether any value $a_{kl}$ in the domain of $z_k$ is consistent with the selected $a_{ij}$ value under the selected constraint $C\left(z_i,z_k\right)$.
* If no value in the domain of $z_k$ is consistent with $a_{ij}$ then $a_{ij}$ cannot be part of a solution of the problem. Therefore remove $a_{ij}$ from $z_i$'s domain.
* Select a new value in some domain and start again until no inconsistent values remain in any domain.

### The domain-array representation

This is essentially the same representation as the previous one. In this, for each variable there is an array that contains all the values for that variable's domain. For every two arrays there is an arc representing a constraint between the two variables.

## Backtracking, backjumping and backmarking
### Basic backtracking
In the OR-Tree representation, basic backtracking is the depth-first left-to-right traversal of the search tree. The next graph shows the backtracking search for the example of the 4-queens problem.

![alt text](https://raw.githubusercontent.com/JoseFPortoles/Notes-on-Constraints-Algorithms/master/grafos/4qbacktracking.png)

Basic backtracking algorithm:
For backtracking to a certain depth $depth$, consider that the domain for $z_{depth}$ is $\{a_{depth,1},\ldots,a_{depth,n_{depth}}\}$ 
* Assign $z_{depth}$ the next value of its domain $a_{depth,k}$
  * Check all constraints $C\left(z_i,z_{depth}\right)$ for $i=1,\ldots,depth-1$
  * If none fails then:
      * If depth is maximum then return $\left(z_1,\ldots,z_{depth}\right)$  
      * If depth is not maximum then recursively backtrack to depth-1
* Back to the first point (assign next value in the domain of $z_{depth}$)


## Backjumping

When checking for constraints at the $z_i$ level of the OR-Tree, it can occur, for instance, that all the $z_i$ domain values fail the tests for $C\left(z_1,z_{i}\right)$ to $C\left(z_{i-2},z_{i}\right)$, so we never get to test the $C\left(z_{i-1},z_{i}\right)$ constraints. In a case like that it makes no sense to backtrack to the previous (i-1) level and start testing again the values of the $z_i$'s domain. Since the values or all the previous variable $z_1,\ldots,z_{i-2}$ are the same, the tesst for the constraints $C\left(z_1,z_{i}\right)$ to $C\left(z_{i-2},z_{i}\right)$ will fail again for all the $z_i$ values. In such a case, the way to avoid redundant computation is to backtrack to level i-2 and continue the search from there. This is called **Backjumping**.

This is only a particular example. In general it does not need to be necessarily constraints $C\left(z_1,z_{i}\right)$ to $C\left(z_{i-2},z_{i}\right)$, but can be any set of constraints in the set $C\left(z_1,z_{i}\right)$ to $C\left(z_{k},z_{i}\right)$ for $k < i-1$

**Trashing** is the redundant computation caused by testing all the values of a variable (in the above case $z_i$) when that variable is not involved in the constraints that fail.

Two additional variables are considered in backjumping:
* **checkdepthk**: Stores the depth at which the constraint checks fail for a particular branch (a particular value of the current variable).
* **jumpback**: records the maximum checkdepth values for all the values of the current variable.  

For instance, if for $z_i=a_{i,l}$ $z_k$ is the deepest variable for which $C\left(z_k,z_{i}\right)$ fails, then k is the **checkdepth** for $a_{i,l}$. While the deepest k over the domain $a_{i,l}$ is the **jumpback** variable of $z_i$.

The idea then is simple: if all the possible values for $z_i$ fail and $C\left(z_k,z_i\right)$ is the deepest constraint that makes one of those values fail, backjump to assign $a_k$ a different value.

## Backmarking

Backjumping doesn't solve all the trashing problem of backtracking. Backjumping focuses in avoiding trashing in cases in which all the constraint checks for a variable in a particular branch fail. However the same problem can arise in branches with successful checks. A constraint $C\left(z_1,z_3\right)$ can give a sequence, say X,V,X,V of check results for the domain values of $z_3$ at a particular branch of $z_2$. When we backtrack to $z_2$ and try its next domain value, we will obtain the same sequence X,V,X,V, given that, while we have changed $z_2$'s value, the results of the sequence are determined by $C\left(z_1,z_3\right)$ which does not depend on $z_2$.

This type of trashing is solved by **tabulation**.

**Tabulation** includes **lemma generation** and **lemma application**

* **Lemma generation**: Keep the records of each constraint check and its result in a table, so each constraint check is a registry on the table.
* **Lemma application**: Each time a new constraint check needs to be done, it is looked up first in the table. If present then the stored result is used. If not present, then the check is computed and added to the table as a new registry.

Tabulation partly solves the problem by avoiding redundant computations but at the cost of building tables, querying for results and it has storage overheads.

An alternative solution is **backmarking**. **Backmarking** avoids computing redundant checks altogether. It uses two variable arrays:
* **Checkdepth(k,l)**: A 2D array For each variable $z_k$ and for each value $a_{kl}$ of the domain of $z_k$ it stores the value of **checkdepthl** used also in backjumping, that is the depth of the deepest constraint check that was performed the last time $z_k$ was assigned $a_{kl}$
* **Backup(k)**: 1D array. On each position $k$ it contains the shallowest depth in the tree that we backtracked to since the last time we visited the values of $z_k$.

For a particular $a_{kl}$ the values of ```checkdepth(k,l)``` and ```backup(k)``` are compared. Comments:
* If the checks for some $a_{kl}$ end up in an V (all constraint tests passed) then $\text{checkdepth}\left(k,l\right) = k-1$
* $\text{backup}\left(k\right)\le{k-1}$ since at least we backtrack to $k-1$
* If $\text{checkdepth}\left(k,l\right)\lt\text{backup}\left(k\right)$ then, following from the previous two points we conclude that last time we checked constraints for $z_k$ values:
  * Checks for $z_k=a_{kl}$ didn't reach depth $k-1$ therefore they must have ended in failure (X).
  * Previous backtracks were below the variables involved in the constraint checks, and therefore those variables hold the same values.
  * Therefore all the constraint checks against $z_k=a_{kl}$ will fail again this time.
* If $\text{checkdepth}\left(k,l\right)\geq\text{backup}\left(k\right)$ then 
  * Constraint checks went at least as deep as the backtrack level $\text{backup}\left(k\right)$ since last time.
  * Then all the levels down to $\text{backup}\left(k\right)-1$ must have passed the tests (V).
  * Since we haven't backtrack as far up as any of the variables involved in those $\text{backup}\left(k\right)-1$ tests then all those tests will be successful this time again.
  
The BackM algorithm, a variant of the Backtr algorithm exploits this properties in order to save time on computations.








